#dataset : Amazon fine food reviews

#Source : https://www.kaggle.com/laowingkin/amazon-fine-food-review-sentiment-analysis

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from string import punctuation
from sklearn import svm
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
from nltk import ngrams
#from itertools import chain
#from wordcloud import WordCloud

In [18]:
# load dataset
df = pd.read_csv('/home/shiv/Dataset/Reviews.csv')

In [17]:
# removing nuetral reviews, with value 3 
#df = df[df['Score'] != 3]
X = df['Text']
#mapping score with poistive and negative
# y_dict = {1:0, 2:0, 4:1, 5:1}
# y = df['Score'].map(y_dict)
def partition(x):
    if x < 3:
        return '1'
    return '0'

temp_score = df.Score

KeyError: 'Text'

In [16]:
df.Score.values

AttributeError: 'Series' object has no attribute 'Score'